In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler,LabelEncoder

# Ma'lumotlarni yuklash (siz o'z ma'lumotlaringizni ishlating)
submission = pd.read_csv("csv/sample_submission.csv")
test = pd.read_csv("csv/test.csv")
train = pd.read_csv("csv/train.csv")

test = test.drop("id",axis=1)

X = train[["Length"]]
y = train[['Age']]

# label_encoder  = LabelEncoder()
# train["Sex"] = label_encoder.fit_transform(train["Sex"])
# train.loc[train["Sex"] == 0, "Sex"] = train["Sex"].mode()

# test["Sex"] = label_encoder.fit_transform(test["Sex"])
# X["Sex"] = label_encoder.fit_transform(X["Sex"])

# Ma'lumotlarni standartlashtirish
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X,y)
X_test_scaled = scaler.transform(test[["Length"]])

# Oddiy Linear Regression
lr_model = LinearRegression()
lr_model.fit(X,y)
lr_pred = lr_model.predict(test[["Length"]])
lr_mae = mean_absolute_error(test[["Length"]], lr_pred)

# Lasso Regression
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X, y)
lasso_pred = lasso_model.predict(test[["Length"]])
lasso_mae = mean_absolute_error(test[["Length"]], lasso_pred)

# Ridge Regression
ridge_model = Ridge(alpha=0.1)
ridge_model.fit(X, y)
ridge_pred = ridge_model.predict(test[["Length"]])
ridge_mae = mean_absolute_error(test[["Length"]], ridge_pred)

print(f"Linear Regression MAE: {lr_mae}")
print(f"Lasso Regression MAE: {lasso_mae}")
print(f"Ridge Regression MAE: {ridge_mae}")

# Eng yaxshi modelni tanlash
best_mae = min(lr_mae, lasso_mae, ridge_mae)
if best_mae == lr_mae:
    best_model = lr_model
elif best_mae == lasso_mae:
    best_model = lasso_model
else:
    best_model = ridge_model

print(f"Eng yaxshi model MAE: {best_mae}")

Linear Regression MAE: 8.590589432632672
Lasso Regression MAE: 8.59543342999965
Ridge Regression MAE: 8.590591681894217
Eng yaxshi model MAE: 8.590589432632672


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import StackingRegressor

# 1. Ma'lumotlarni yuklash
submission = pd.read_csv("csv/sample_submission.csv")
test = pd.read_csv("csv/test.csv")
train = pd.read_csv("csv/train.csv")

test = test.drop("id",axis=1)

X = train.drop(columns=["Age", "id"])
y = train[['Age']]

label_encoder  = LabelEncoder()
train["Sex"] = label_encoder.fit_transform(train["Sex"])
train.loc[train["Sex"] == 0, "Sex"] = train["Sex"].mode()

test["Sex"] = label_encoder.fit_transform(test["Sex"])
X["Sex"] = label_encoder.fit_transform(X["Sex"])

# 2. Ma'lumotlarni bo'lish

# 3. Base modellarni yaratish
base_models = [
    ('ridge', Ridge(alpha=1.0)),
    ('lasso', Lasso(alpha=0.1)),
    ('rf', RandomForestRegressor(n_estimators=50, random_state=42))
]

# 4. Meta-model yaratish
meta_model = LinearRegression()

# 5. Stack Regressionni yaratish
stack_model = StackingRegressor(
    estimators=base_models, 
    final_estimator=meta_model
)

# 6. Modelni o'qitish
stack_model.fit(X, y)

# 7. Natijalarni baholash
y_pred = stack_model.predict(test)
submission["Age"] = y_pred
submission.to_csv("my_submission.csv",index=False)

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_stacking.py:970: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
